## Install Dependencies

In [1]:
!pip install qrcode
!pip install fpdf

## import csv file

In [52]:
import pandas as pd
data = pd.read_csv('Adressaten.csv', sep=';', header=0)

In [63]:
data

,Adressat,Typ,Zugehörige Behörde,Zuständige Person,Anzahl Modelle,Kommentar,Hash
0,Amt für IT und Digitalisierung,Amt,Senatskanzlei,Mathias Wieckmann,1,NaN,KDJOM
1,Amt für Bauordnung und Hochbau,Amt,Behörde für Stadtentwicklung und Wohnen,Herr Koch,1,NaN,KLNSN
2,Wohnen,Abteilung,Behörde für Stadtentwicklung und Wohnen,Frau Dr. Opitz,1,NaN,GJSCN
3,Integrierte Stadtteilentwicklung,Abteilung,Behörde für Stadtentwicklung und Wohnen,Frau Vorkoeper,1,NaN,UNVNF
4,Bodenordnung,Abteilung,Behörde für Stadtentwicklung und Wohnen,Herr Pletz,1,NaN,LAGWY
...,...,...,...,...,...,...,...
83,Stadtwerkstatt,Abteilung,Behörde für Stadtentwicklung und Wohnen,Herr Lieven,2,NaN,VDUFB
84,Digitalisierung und Steuerung,Abteilung,"Landesbetrieb Straßen, Brücken und Gewässer",Herr Heel,3,An die Referate Strategie.Werkstatt; DigiLab; ...,BMPNB
85,Straßen,Abteilung,"Landesbetrieb Straßen, Brücken und Gewässer",Herr Hansen,2,Insb. Planung Infrastruktur ÖPNV und Planung I...,NWQVH
86,Gewässer und Hochwasserschutz,Abteilung,"Landesbetrieb Straßen, Brücken und Gewässer",Dr. Olaf Müller,2,Inbs. An Hydrologie und Wasserwirtschaft; Stur...,KIODX


In [54]:
data[data["Anzahl Modelle "] > 1]

,Adressat,Typ,Zugehörige Behörde,Zuständige Person,Anzahl Modelle,Kommentar,Hash
5,Stadtwerkstatt,Abteilung,Behörde für Stadtentwicklung und Wohnen,Herr Lieven,2,NaN,VDUFB
56,Digitalisierung und Steuerung,Abteilung,"Landesbetrieb Straßen, Brücken und Gewässer",Herr Heel,3,An die Referate Strategie.Werkstatt; DigiLab; ...,BMPNB
57,Straßen,Abteilung,"Landesbetrieb Straßen, Brücken und Gewässer",Herr Hansen,2,Insb. Planung Infrastruktur ÖPNV und Planung I...,NWQVH
58,Gewässer und Hochwasserschutz,Abteilung,"Landesbetrieb Straßen, Brücken und Gewässer",Dr. Olaf Müller,2,Inbs. An Hydrologie und Wasserwirtschaft; Stur...,KIODX


In [55]:
data = data.append(data[data["Anzahl Modelle "] > 1], ignore_index=True)

In [56]:
data = data.append(data[data["Hash"] == "BMPNB"].iloc[0], ignore_index=True)

In [23]:
data["Typ"].unique()

array(['Amt', 'Abteilung', 'Stabstelle', 'Landeslabor', 'Dezernat'],
      dtype=object)

In [57]:
# Check for duplicates
print(len(data["Hash"]))
print(len(data["Hash"].unique()))

88
83


## generate QR Codes + save as images

In [59]:
import qrcode

url = 'https://studie.cut.hcu-hamburg.de/?h='
for i in range(len(data)):
    string = url+data.values[i,6]
    img = qrcode.make(string)
    type(img)  # qrcode.image.pil.PilImage
    img.save("qr_codes/{}_qr_code.png".format(data.values[i,6]))

## generate PDF with QR Codes and URL
Layout is suitable for Zweckform 55x55 mm 
https://www.avery-zweckform.com/blanko-etiketten/quadratisch-55x55-mm

In [4]:
from fpdf import FPDF


In [120]:
def getGreeting(row):
    if row["Typ"] == "Abteilung":
        return ("Abteilung " + row["Adressat"] )
    elif row["Typ"] == "Stabsstelle":
        return ("Stabsstelle  " + row["Adressat"])
    else: # row["Typ"] == "Amt" or row["Typ"] == "Dezernat":
        return(row["Adressat"])

In [121]:
getGreeting(data.loc[1])

'Amt für Bauordnung und Hochbau'

In [122]:

class PDF(FPDF):
    def qr(self, data):
        url = 'https://studie.cut.hcu-hamburg.de'
        self.add_font(family="Century Gothic", fname = 'Century Gothic.ttf', uni = True)
        self.add_font(family="Century Gothic Bold", fname = 'Century Gothic Bold.ttf', uni = True)
        self.set_font('Century Gothic', size = 8)
        self.set_auto_page_break(False)
        run_list = range(0,len(data),15)
        for j in run_list:
            print(j)
            self.add_page()
            for i in range(0, 15,3):
                if i == 0:
                    y = 8 
                    try:
                        self.image('qr_codes/{}_qr_code.png'.format(data.values[j+i,6]), 31, y, 35)
                        self.set_y(45)
                        self.set_x(25)
                        self.set_font('Century Gothic Bold', size = 8)
                        self.multi_cell(w=45, h=3, txt=getGreeting(data.loc[j+i]), align="C")
                        self.set_y(52)
                        self.set_x(25)
                        self.set_font('Century Gothic', size = 7)
                        self.multi_cell(w=45, h=2.2, txt=data.loc[j+i]["Zugehörige Behörde"], align="C")

                        #self.text(23, y+45, url)
                        #self.cell(60, txt=getGreeting(data.loc[j+i]), align="C")
                        #self.text(30, y+40, getGreeting(data.loc[j+i]))
                        #self.text(35, y+45, data.loc[j+i]["Zugehörige Behörde"])
                    except:
                        None
                    try:
                        self.image('qr_codes/{}_qr_code.png'.format(data.values[j+i+1,6]), 88, y, 35)
                        self.set_y(45)
                        self.set_x(82)
                        self.set_font('Century Gothic Bold', size = 8)
                        self.multi_cell(w=45, h=3, txt=getGreeting(data.loc[j+i+1]), align="C")
                        self.set_y(52)
                        self.set_x(82)
                        self.set_font('Century Gothic', size = 7)
                        self.multi_cell(w=45, h=2.2, txt=data.loc[j+i+1]["Zugehörige Behörde"], align="C")
                        #self.text(80, y+45, url)

                    except:
                        None
                    try:
                        self.image('qr_codes/{}_qr_code.png'.format(data.values[j+i+2,6]), 145, y, 35)
                        self.set_y(45)
                        self.set_x(140)
                        self.set_font('Century Gothic Bold', size = 8)
                        self.multi_cell(w=45, h=3, txt=getGreeting(data.loc[j+i+2]), align="C")
                        self.set_y(52)
                        self.set_x(140)
                        self.set_font('Century Gothic', size = 7)
                        self.multi_cell(w=45, h=2.2, txt=data.loc[j+i+2]["Zugehörige Behörde"], align="C")
                        #self.text(138, y+45, url)

                    except:
                        None
                else:
                    y = y + 57
                    try:
                        self.image('qr_codes/{}_qr_code.png'.format(data.values[j+i,6]), 31, y, 35)
                        #self.text(23, y+45, url)
                        self.set_y(y + 35)
                        self.set_x(25)
                        self.set_font('Century Gothic Bold', size = 8)
                        self.multi_cell(w=45, h=3, txt=getGreeting(data.loc[j+i]), align="C")
                        self.set_y(y + 42)
                        self.set_x(25)
                        self.set_font('Century Gothic', size = 7)
                        self.multi_cell(w=45, h=2.2, txt=data.loc[j+i]["Zugehörige Behörde"], align="C")
                    except:
                        None
                    try:
                        self.image('qr_codes/{}_qr_code.png'.format(data.values[j+i+1,6]), 88, y, 35)
                        #self.text(80, y+45, url)
                        self.set_y(y + 35)
                        self.set_x(82)
                        self.set_font('Century Gothic Bold', size = 8)
                        self.multi_cell(w=45, h=3, txt=getGreeting(data.loc[j+i+1]), align="C")
                        self.set_y(y + 42)
                        self.set_x(82)
                        self.set_font('Century Gothic', size = 7)
                        self.multi_cell(w=45, h=2.2, txt=data.loc[j+i+1]["Zugehörige Behörde"], align="C")
                    except:
                        None
                    try:
                        self.image('qr_codes/{}_qr_code.png'.format(data.values[j+i+2,6]), 145, y, 35)
                        #self.text(138, y+45, url)
                        self.set_y(y + 35)
                        self.set_x(140)
                        self.set_font('Century Gothic Bold', size = 8)
                        self.multi_cell(w=45, h=3, txt=getGreeting(data.loc[j+i+2]), align="C")
                        self.set_y(y + 42)
                        self.set_x(140)
                        self.set_font('Century Gothic', size = 7)
                        self.multi_cell(w=45, h=2.2, txt=data.loc[j+i+2]["Zugehörige Behörde"], align="C")
                    except:
                        None




# Instantiation of inherited class
pdf = PDF()
pdf.alias_nb_pages()
pdf.qr(data)
pdf.output('qr_codes.pdf', 'F')


0
15
30
45
60
75


''

In [18]:
print(len(data))

83


In [123]:
def getGreeting(row):
    if row["Typ"] == "Abteilung":
        return ("An:\n"+"Abteilung " + row["Adressat"] )
    elif row["Typ"] == "Stabsstelle":
        return ("An:\n"+"Stabsstelle  " + row["Adressat"])
    else: # row["Typ"] == "Amt" or row["Typ"] == "Dezernat":
        return("An:\n"+row["Adressat"])

In [128]:

class PDF(FPDF):
    
    def qr(self, data):
        url = 'http://studie.cut.hcu-hamburg.de/?h='
        self.add_font(family="Century Gothic", fname = 'Century Gothic.ttf', uni = True)
        self.add_font(family="Century Gothic Bold", fname = 'Century Gothic Bold.ttf', uni = True)
        self.set_font('Century Gothic', size = 7)
        self.set_auto_page_break(False)
        run_list = range(0,len(data),15)
        for j in run_list:
            print(j)
            self.add_page()
            for i in range(0, 15,3):
                if i == 0:
                    y = 8
                    try:
                        self.set_y(20)
                        self.set_x(25)
                        self.set_font('Century Gothic Bold', size = 12)
                        self.multi_cell(w=45, h=3, txt=data.loc[j+i]["Hash"], align="C")
                        
                        self.set_y(30)
                        self.set_x(25)
                        self.set_font('Century Gothic', size = 8)
                        self.multi_cell(w=45, h=3, txt=getGreeting(data.loc[j+i])+"\n"+data.loc[j+i]["Zugehörige Behörde"] + "\nFreie und Hansestadt Hamburg", align="L")

                    except:
                        None
                    try:
                        self.set_y(20)
                        self.set_x(82)
                        self.set_font('Century Gothic Bold', size = 12)
                        self.multi_cell(w=45, h=3, txt=data.loc[j+i+1]["Hash"], align="C")
                        
                        self.set_y(30)
                        self.set_x(82)
                        self.set_font('Century Gothic', size = 8)
                        self.multi_cell(w=45, h=3, txt=getGreeting(data.loc[j+i+1])+"\n"+data.loc[j+i+1]["Zugehörige Behörde"] + "\nFreie und Hansestadt Hamburg", align="L")

                    except:
                        None
                    try:
                        self.set_y(20)
                        self.set_x(140)
                        self.set_font('Century Gothic Bold', size = 12)
                        self.multi_cell(w=45, h=3, txt=data.loc[j+i+2]["Hash"], align="C")

                        self.set_y(30)
                        self.set_x(140)
                        self.set_font('Century Gothic', size = 8)
                        self.multi_cell(w=45, h=3, txt=getGreeting(data.loc[j+i+2])+"\n"+data.loc[j+i+2]["Zugehörige Behörde"] + "\nFreie und Hansestadt Hamburg", align="L")

                    except:
                        None
                else:
                    y = y + 57
                    try:
                        self.set_y(y + 20)
                        self.set_x(25)
                        self.set_font('Century Gothic Bold', size = 12)
                        self.multi_cell(w=45, h=3, txt=data.loc[j+i]["Hash"], align="C")

                        self.set_y(y + 30)
                        self.set_x(25)
                        self.set_font('Century Gothic', size = 8)
                        self.multi_cell(w=45, h=3, txt=getGreeting(data.loc[j+i])+"\n"+data.loc[j+i]["Zugehörige Behörde"] + "\nFreie und Hansestadt Hamburg", align="L")
                    except:
                        None
                    try:
                        self.set_y(y + 20)
                        self.set_x(82)
                        self.set_font('Century Gothic Bold', size = 12)
                        self.multi_cell(w=45, h=3, txt=data.loc[j+i+1]["Hash"], align="C")

                        self.set_y(y + 30)
                        self.set_x(82)
                        self.set_font('Century Gothic', size = 8)
                        self.multi_cell(w=45, h=3, txt=getGreeting(data.loc[j+i+1])+"\n"+data.loc[j+i+1]["Zugehörige Behörde"] + "\nFreie und Hansestadt Hamburg", align="L")
                    except:
                        None
                    try:
                        self.set_y(y + 20)
                        self.set_x(140)
                        self.set_font('Century Gothic Bold', size = 12)
                        self.multi_cell(w=45, h=3, txt=data.loc[j+i+2]["Hash"], align="C")

                        self.set_y(y + 30)
                        self.set_x(140)
                        self.set_font('Century Gothic', size = 8)
                        self.multi_cell(w=45, h=3, txt=getGreeting(data.loc[j+i+2])+"\n"+data.loc[j+i+2]["Zugehörige Behörde"] + "\nFreie und Hansestadt Hamburg", align="L")
                    except:
                        None




# Instantiation of inherited class
pdf = PDF()
pdf.alias_nb_pages()
pdf.qr(data)
pdf.output('codes.pdf', 'F')


0
15
30
45
60
75


''

In [132]:
run_list = range(0,len(data))
for j in run_list:
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87


In [133]:

class PDF(FPDF):
    
    def qr(self, data):
        url = 'http://studie.cut.hcu-hamburg.de/?h='
        self.add_font(family="Century Gothic", fname = 'Century Gothic.ttf', uni = True)
        self.add_font(family="Century Gothic Bold", fname = 'Century Gothic Bold.ttf', uni = True)
        self.set_font('Century Gothic', size = 9)
        self.set_auto_page_break(False)
        run_list = range(0,len(data))
        for j in run_list:
            print(j)
            self.add_page()
            self.set_y(65)
            self.set_x(28)
            self.multi_cell(w=70, h=3, txt=getGreeting(data.loc[j])+"\n"+data.loc[j]["Zugehörige Behörde"] + "\nFreie und Hansestadt Hamburg", align="L")



# Instantiation of inherited class
pdf = PDF()
pdf.alias_nb_pages()
pdf.qr(data)
pdf.output('anschreiben.pdf', 'F')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87


''